In [1]:
import transformers, torch
import polars as pl
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset, load_from_disk
from dotenv import load_dotenv

%load_ext dotenv
%dotenv

load_dotenv()

True

In [2]:
load_dataset("stanfordnlp/imdb").save_to_disk("data/imdb")
ds = load_from_disk("data/imdb")
train_df = ds["train"].to_polars()
test_df = ds["test"].to_polars()
unsupervised_df = ds["unsupervised"].to_polars()

Saving the dataset (0/1 shards):   0%|          | 0/25000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/25000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:
train_df = train_df.with_columns([pl.col("label").cast(pl.Boolean)])
test_df = test_df.with_columns([pl.col("label").cast(pl.Boolean)])
unsupervised_df = unsupervised_df.with_columns([pl.col("label").cast(pl.Boolean)])

In [7]:
train_dataloader = DataLoader(dataset=train_df)
test_dataloader = DataLoader(dataset=test_df)

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer

tokenizer_base = AutoTokenizer.from_pretrained("./gemma-3-1b-pt-tokenizer")
model_base = AutoModelForCausalLM.from_pretrained("./gemma-3-1b-pt-model")

In [12]:
generation_config = transformers.GenerationConfig(
    max_new_tokens=100,
    max_time=10,
    do_sample=False,
    temperature=0.0,
)
prompt = """
Happiness is a butterfly
Try to catch it, like, every night
It escapes from my hands into moonlight
"""
input_ids = tokenizer_base.encode(prompt, return_tensors="pt", truncation=True)
outputs = model_base.generate(
    inputs=input_ids,
    generation_config=generation_config,
    output_scores=True,
    return_dict_in_generate=True,
    tokenizer=tokenizer_base,
)

print(tokenizer_base.decode(outputs.sequences[0], skip_special_tokens=True))


Happiness is a butterfly
Try to catch it, like, every night
It escapes from my hands into moonlight
And I'm not sure if I'll ever get it back
But I'll keep trying
I'll keep trying
I'll keep trying
I'll keep trying
I'll keep trying
I
